In [1]:
import ee
import geemap

# Inicializar la biblioteca Earth Engine
ee.Initialize()

In [2]:
# Definir la región de interés (ROI)
roi = ee.Geometry.Polygon(
    [[[-122.45, 37.74],
      [-122.45, 37.8],
      [-122.4, 37.8],
      [-122.4, 37.74]]])



In [3]:
# Función para calcular el GCVI
def calculate_gcvi(image):
    gcvi = image.expression(
        'NIR / GREEN - 1', {
            'NIR': image.select('SR_B5'),  # NIR band
            'GREEN': image.select('SR_B3')  # Green band
        }).rename('GCVI')
    return image.addBands(gcvi)



In [4]:
# Filtrar la colección de imágenes
collection = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
              .filterBounds(roi)
              .filterDate('2017-01-01', '2017-12-31')
              .map(calculate_gcvi)
              .select('GCVI')
              .sort('system:time_start'))

In [5]:
# Crear el composite harmonico
harmonicLandsat = collection.map(lambda img: img.set('date', img.date().format('YYYY-MM-dd')))


In [6]:
# Definir las frecuencias y términos de Fourier
harmonics = 3
harmonics_frequencies = ee.List([1, 2, 3, 4])

In [10]:
def harmonic_terms(image):
    date = ee.Date(image.get('system:time_start'))
    years = ee.Date('1970-01-01').difference(date, 'year')
    timeRadians = ee.Image.constant(years.multiply(2 * 3.141592))
    
    # Calcular términos armónicos
    harmonics_list = harmonics_frequencies.map(lambda f:
                                               timeRadians.multiply(ee.Number(f)).cos().rename('cos_' + ee.Number(f).format())
                                               .addBands(timeRadians.multiply(ee.Number(f)).sin().rename('sin_' + ee.Number(f).format())))
    
    return image.addBands(ee.Image.cat(harmonics_list))

In [11]:
# Mapear la función a la colección de imágenes
harmonic_collection = harmonicLandsat.map(harmonic_terms)



TypeError: can only concatenate str (not "String") to str

In [ ]:
# Función para aplicar la regresión y calcular R² y RMSE
def fit_harmonic_model(image):
    fitted = image.select(['cos_1', 'sin_1']).reduce(ee.Reducer.linearFit())
    residuals = image.subtract(fitted)
    rmse = residuals.pow(2).reduce(ee.Reducer.mean()).sqrt().rename('RMSE')
    r_squared = fitted.select('scale').pow(2).rename('R_squared')
    return image.addBands(fitted).addBands(rmse).addBands(r_squared)


In [ ]:
# Aplicar el modelo
fitted_harmonic_img = fit_harmonic_model(harmonicCoefficients)

In [ ]:
# Visualización en el mapa
Map = geemap.Map(center=[37.76, -122.44], zoom=10)

In [ ]:
# Visualizar los coeficientes armónicos
Map.addLayer(fitted_harmonic_img.select(['offset']), {'min': -0.5, 'max': 0.5, 'palette': ['blue', 'white', 'red']}, 'Offset')
Map.addLayer(fitted_harmonic_img.select(['scale']), {'min': 0, 'max': 1, 'palette': ['black', 'white']}, 'Scale')
Map.addLayer(fitted_harmonic_img.select(['RMSE']), {'min': 0, 'max': 0.5, 'palette': ['black', 'red']}, 'RMSE')
Map.addLayer(fitted_harmonic_img.select(['R_squared']), {'min': 0, 'max': 1, 'palette': ['white', 'green']}, 'R_squared')

# Mostrar el mapa
Map.addLayer(roi, {}, 'ROI')
Map.addLayerControl()
Map

In [15]:
import ee
import geemap

# Inicializar la biblioteca Earth Engine
ee.Initialize()

# Definir la región de interés (ROI)
roi = ee.Geometry.Polygon(
    [[[-122.45, 37.74],
      [-122.45, 37.8],
      [-122.4, 37.8],
      [-122.4, 37.74]]])

# Función para calcular el GCVI
def calculate_gcvi(image):
    gcvi = image.expression(
        'NIR / GREEN - 1', {
            'NIR': image.select('SR_B5'),  # NIR band
            'GREEN': image.select('SR_B3')  # Green band
        }).rename('GCVI')
    return image.addBands(gcvi)

# Filtrar la colección de imágenes
collection = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
              .filterBounds(roi)
              .filterDate('2017-01-01', '2017-12-31')
              .map(calculate_gcvi)
              .select('GCVI')
              .sort('system:time_start'))

# Crear el composite harmonico
harmonicLandsat = collection.map(lambda img: img.set('date', img.date().format('YYYY-MM-dd')))

# Definir las frecuencias y términos de Fourier
harmonics = 3
harmonics_frequencies = ee.List.sequence(1, harmonics)

# Definir la función que aplica la transformada de Fourier
def harmonic_terms(image):
    date = ee.Date(image.get('system:time_start'))
    years = ee.Date('1970-01-01').difference(date, 'year')
    timeRadians = ee.Image.constant(years.multiply(2 * 3.141592))
    
    def compute_harmonics(f):
        f = ee.Number(f)
        cos_band = timeRadians.multiply(f).cos().rename(f'cos_{f.getInfo()}')
        sin_band = timeRadians.multiply(f).sin().rename(f'sin_{f.getInfo()}')
        return image.addBands(cos_band).addBands(sin_band)
    
    harmonic_images = harmonics_frequencies.map(compute_harmonics)
    harmonic_image = ee.Image(harmonic_images.iterate(lambda img, result: ee.Image(result).addBands(img), ee.Image()))
    
    return image.addBands(harmonic_image)

# Mapear la función a la colección de imágenes
harmonic_collection = harmonicLandsat.map(harmonic_terms)

# Unir todos los términos en una sola imagen
harmonic_img = harmonic_collection.toArray()

# Preparar los coeficientes
coefficients = harmonic_img.arraySolve(ee.Image.constant(harmonics))

# Extraer los coeficientes
harmonicCoefficients = ee.Image(coefficients)

# Función para aplicar la regresión y calcular R² y RMSE
def fit_harmonic_model(image):
    fitted = image.select(['cos_1', 'sin_1']).reduce(ee.Reducer.linearFit())
    residuals = image.subtract(fitted)
    rmse = residuals.pow(2).reduce(ee.Reducer.mean()).sqrt().rename('RMSE')
    r_squared = fitted.select('scale').pow(2).rename('R_squared')
    return image.addBands(fitted).addBands(rmse).addBands(r_squared)

# Aplicar el modelo
fitted_harmonic_img = fit_harmonic_model(harmonicCoefficients)

# Visualización en el mapa
Map = geemap.Map(center=[37.76, -122.44], zoom=10)

# Visualizar los coeficientes armónicos
Map.addLayer(fitted_harmonic_img.select(['offset']), {'min': -0.5, 'max': 0.5, 'palette': ['blue', 'white', 'red']}, 'Offset')
Map.addLayer(fitted_harmonic_img.select(['scale']), {'min': 0, 'max': 1, 'palette': ['black', 'white']}, 'Scale')
Map.addLayer(fitted_harmonic_img.select(['RMSE']), {'min': 0, 'max': 0.5, 'palette': ['black', 'red']}, 'RMSE')
Map.addLayer(fitted_harmonic_img.select(['R_squared']), {'min': 0, 'max': 1, 'palette': ['white', 'green']}, 'R_squared')

# Mostrar el mapa
Map.addLayer(roi, {}, 'ROI')
Map.addLayerControl()
Map


EEException: A mapped function's arguments cannot be used in client-side operations